Part 4

For part 4 of the project, you will be using your MySQL database from part 3 to answer meaningful questions for your stakeholder. They want you to use your hypothesis testing and statistics knowledge to answer 3 questions about what makes a successful movie.

Questions to Answer

The stakeholder's first question is: does the MPAA rating of a movie (G/PG/PG-13/R) affect how much revenue the movie generates?

They want you to perform a statistical test to get a mathematically-supported answer.
They want you to report if you found a significant difference between ratings.
If so, what was the p-value of your analysis?
And which rating earns the most revenue?
They want you to prepare a visualization that supports your finding.

It is then up to you to think of 2 additional hypotheses to test that your stakeholder may want to know.

Some example hypotheses you could test:

Do movies that are over 2.5 hours long earn more revenue than movies that are 1.5 hours long (or less)?

Do movies released in 2020 earn less revenue than movies released in 2018?

How do the years compare for movie ratings?

Do some movie genres earn more revenue than others?

Are some genres higher rated than others?
etc.

Specifications

Your Data

A critical first step for this assignment will be to retrieve additional movie data to add to your SQL database.

You will want to use the TMDB API again and extract data for additional years.
You may want to review the optional lesson from Week 1 on "Using Glob to Load Many Files" to load and combine all of your API results for each year.
However, trying to extract the TMDB data for all movies from 2000-2022 could take >24 hours!

To address this issue, you should EITHER:
Define a smaller (but logical) period of time to use for your analyses (e.g., last 10 years, 2010-2019 (pre-pandemic, etc).

OR coordinate with cohort-mates and divide the API calls so that you can all download the data for a smaller number of years and then share your downloaded JSON data.


Deliverables

You should use the same project repository you have been using for Parts 1-3 (for your portfolio).

Create a new notebook in your project repository just for the hypothesis testing (like "Part 4 - Hypothesis Testing.ipynb")

Make sure the results and visualization for all 3 hypotheses are in your notebook.
Please submit the link to your GitHub repository for this assignment.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from scipy import stats

import warnings
warnings.filterwarnings('ignore')

<font size="5">Connect to TMDB through API using JSON<font>

In [ ]:
#Load API Credentials
import json
with open('/Users/benjaminengel/Documents/.Secret/tmdb_api.json') as f:
    login = json.load(f)
login.keys()

In [ ]:
#Install tqdm
!pip install tqdm

In [ ]:
# Install tmdbsimple
!pip install tmdbsimple

In [ ]:
import tmdbsimple as tmdb
tmdb.API_KEY =  login['API Key']

In [ ]:
import os
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

In [ ]:
basics=pd.read_csv("Data/title_basics.csv.gz", low_memory = False)
basics

In [ ]:
#From the LP
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

In [ ]:
def get_movie_with_rating(movie_id):
    """Copied from Coding Dojo Learning Platform"""
    # Get the movie object for the current id
    movie = tmdb.Movies(movie_id)
    # save the .info .releases dictionaries
    info = movie.info()
    releases = movie.releases()
    # Loop through countries in releases
    for c in releases['countries']:
        # if the country abbreviation==US
        if c['iso_3166_1' ] =='US':
            ## save a "certification" key in the info dict with the certification
            info['certification'] = c['certification']
    
    return info

In [ ]:
#Make object for years needed.
YEARS_TO_GET = [2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]

In [ ]:
#Make object for Errors.
errors = []

In [ ]:
from tqdm import tqdm_notebook
# Start of OUTER loop
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):

    #Defining the JSON file to store results for year
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'

    # Check if file exists
    file_exists = os.path.isfile(JSON_FILE)
    # If it does not exist: create it
    if file_exists == False:
        # save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)

    #Saving new year as the current df
    df = basics.loc[ basics['startYear']==YEAR].copy()
    # saving movie ids to list
    movie_ids = df['tconst'].copy()

    # Load existing data from json into a dataframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)

    # filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]



    #Get index and movie id from list
    # INNER Loop
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        try:
            # Retrieve then data for the movie id
            temp = get_movie_with_rating(movie_id)  
            # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
            
        except Exception as e:
            errors.append([movie_id, e])
            
    #Save the year's results as csv.gz file
    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz",
                         compression="gzip", index=False)

#Print number of errors
print(f"- Total errors: {len(errors)}")

In [ ]:
data2010 = pd.read_csv('Data/final_tmdb_data_2010.csv.gz')
data2011 = pd.read_csv('Data/final_tmdb_data_2011.csv.gz')
data2012 = pd.read_csv('Data/final_tmdb_data_2012.csv.gz')
data2013 = pd.read_csv('Data/final_tmdb_data_2013.csv.gz')
data2014 = pd.read_csv('Data/final_tmdb_data_2014.csv.gz')
data2015 = pd.read_csv('Data/final_tmdb_data_2015.csv.gz')
data2016 = pd.read_csv('Data/final_tmdb_data_2016.csv.gz')
data2017 = pd.read_csv('Data/final_tmdb_data_2017.csv.gz')
data2018 = pd.read_csv('Data/final_tmdb_data_2018.csv.gz')
data2019 = pd.read_csv('Data/final_tmdb_data_2019.csv.gz')

In [ ]:
df= pd.concat([data2010, data2011, data2012, data2013, data2014, data2015, data2016, data2017, data2018, data2019])
df.head()

**Question 1: Does the MPAA rating of a movie (G/PG/PG-13/R) affect how much revenue the movie generates?**

Null Hypothesis: There is no difference in movie revenue based on the movies MPAA rating.

Alternative Hypothesis: There is a difference in movie revenue depending on the movies MPAA rating.

In [ ]:
#Making two dataframe for "smoker" and for "non-smoker"
g_df = df.loc[df['smoker']=='yes',:].copy()
pg_df = df.loc[df['smoker']=='yes',:].copy()
pg13_df = df.loc[df['smoker']=='yes',:].copy()
r_df = df.loc[df['smoker']=='yes',:].copy()
#Define our feature of interest
smoker_charges = smoker_df['charges']
nonsmoker_charges = nonsmoker_df['charges']

**Size of the two groups.**

In [ ]:
print(f'Size of smoker {len(smoker_charges)}')
print(f'Size of non_smoker {len(nonsmoker_charges)}')

**Testing Assumptions**

In [ ]:
#Check for outliers in smoker_df
zscores= stats.zscore(smoker_charges)
outliers = abs(zscores)>3
np.sum(outliers)

**No outliers in Smoker**

In [ ]:
#Check for outliers in nonsmoker_df
zscores= stats.zscore(nonsmoker_charges)
outliers = abs(zscores)>3
np.sum(outliers)

In [ ]:
#Looping out all the outliers
while np.sum(outliers) > 0 and len(outliers)> 0:
    nonsmoker_charges = nonsmoker_charges[(np.abs(stats.zscore(nonsmoker_charges)) < 3)]
    zscores= stats.zscore(nonsmoker_charges)
    outliers = abs(zscores)>3
    np.sum(outliers)

In [ ]:
#Recheck for outliers in nonsmoker_df
zscores= stats.zscore(nonsmoker_charges)
outliers = abs(zscores)>3
np.sum(outliers)

**Normality**

In [ ]:
#Test the smoker_charges group for normality
result_smoker_charges = stats.normaltest(smoker_charges)
result_smoker_charges

In [ ]:
Smoker charges are not normally distributed. The samples sizes is large enough to not need to be normally distributed.

**Equal Variance**

In [ ]:
#Test for equal variance
result = stats.levene(smoker_charges, nonsmoker_charges)
result

In [ ]:
Variances is not equal. We'll have to put "equal_var = False" when performing our t-test.

In [ ]:
Perform and evaluate the t-test

In [ ]:
result = stats.ttest_ind(smoker_charges, nonsmoker_charges, equal_var = False)
result

Our p-value < alpha (0.05). I reject the null hypothesis and accept that there is a significant difference between smoker and non-smoker insurance charges.

**Supporting Visualization**

In [ ]:
ax = sns.barplot(x=['smoker','non-smoker'], y=[smoker_charges.mean(),nonsmoker_charges.mean()])
ax.bar_label(ax.containers[0])
plt.title('Smoker Mean vs. Non-smoker Mean Charges');

**Question 2: Do movies released in 2011 earn less revenue than movies released in 2019?**

Null Hypothesis: There is no difference in movie revenue based on the movies release date. 2011 vs 2019.

Alternative Hypothesis: There is a difference in movie revenue depending on the movies release date. 2011 vs 2019.

**Question 3: Do Do some movie genres earn more revenue than others?**

Null Hypothesis: There is no difference in movie revenue based on the movies genre.

Alternative Hypothesis: There is a difference in movie revenue depending on the movies genre.